In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
!pip install tweepy

### Twitter Erişim

In [2]:
import tweepy, codecs


consumer_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'
consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Veri Çekme

In [4]:
from  openpyxl import *
unlist=["beinsport", "arena", "galatasaray","maç","stat", "stadyum", "turktelekom_tk", "bahis", "basketbol", "futbol", "voleybol", "@haberglobal"]
book = load_workbook("twitter_data.xlsx")
sheet=book.active
sheet_end=sheet.max_row
tags =["ttnet", "turktelekom","türk telekom", "silsupur","silsüpür", "avea", "tivibu"]
for tag in tags:
    results = api.search ( q = tag, lang = "tr", result_type = "recent", count = 1000, tweet_mode='extended')
    for doc in results:  
        c = True
        for row in sheet.iter_rows(min_row=2, min_col=2 ,max_row = sheet_end, max_col = 2):
            for cell in row:
                if cell.value == doc.full_text:
                    c = False
                    break     
        if c: 
            if (doc.author.screen_name !="TTDestek" ):
                if any(item in doc.full_text.lower() for item in unlist) != True :
                    sheet.append([doc.id, doc.full_text, doc.created_at, doc.favorite_count ,doc.retweet_count, doc.author.screen_name, doc.author.followers_count, doc.author.location,tag])
book.save("twitter_data.xlsx")
book.close()
    

### Data Cleaning

In [ ]:
#Define Preprocessing Functions

#importing libraries
from TurkishStemmer import TurkishStemmer
import re

def lowLetters(tweet):
    return tweet.lower()

def applyStemmer(word):
    if word in kelime_kok_sozluk.keys():   
        return kelime_kok_sozluk[word].split(":")[0]
    else:
        return TurkishStemmer().stem(word)

#cleaning tweets
def cleanTweet(tweet):
    #removing usernames
    tweet = re.sub('@[^\s]+', '', tweet)
    #removing hashtags
    tweet = re.sub(r'#[^\s]+', '', tweet)
    #removing links
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)
    #removing numeric characters
    tweet = ["" if i.isnumeric() else i for i in tweet]
    tweet = "".join(tweet)
    #removing punctuation
    tweet = re.sub(r'[^\w\s]',' ', tweet)
    #removing single characters
    tweet = re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)
    #removing emojis
    tweet = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE).sub(r'', tweet)
    return tweet